# Filtering ACMA Register of Radiocommuncation Licences

The Register of Radiocommunication Licences (RRL) is the publicly available database of radiocommunication licences, location and frequency ranges. However, the vastness of the available data presents challenges to extracting useful data for any given project.

The code below will isolate data relevant to mid-band spectrum usage with a primary focus on Wireless Broadband Access (WBA). Typically, spectrum used for WBA is allocated in a spectrum licence which are multidimensional.

### Spectrum Licence Elements:
- Frequency range(s)
- Geographical location
- Client

The purpose of the code below is provide data in a format which is compatiable with an interactive treemap using squarify or plotly packages.

## RRL Complications

The data required for visualisation exists in two files: auth_spectrum_freq.csv and licence.csv. While licence.csv contains information regarding the licence itself, it does not contain geographic or spectrum usage information. The code uses licence.csv to filter out every licence that is not a spectrum licence. 

> NOTE: There are apparatus licences used by NBN Co. for WBA.

The auth_spec_freq.csv file is not compatiable with licence.csv seeing as there are multiple licence number entires whereas licence.csv contains only one unique entry per licence number. As a result, the licence.csv dataset is being used a key data set to the frequency value dataset.


## Identify Spectrum Licences:

In [6]:
import pandas as pd
import numpy as np

lic = pd.read_csv('licence.csv')

# Remove unecessary columns
lic = lic.drop(columns=['SV_ID', 'SS_ID', 'DATE_ISSUED', 'DATE_OF_EFFECT', 'DATE_OF_EXPIRY', 'STATUS', 'STATUS_TEXT', 'AP_ID', 'SHIP_NAME', 'BSL_NO', 'AP_PRJ_IDENT'])

# Isolate spectrum licences and reset index
lic = lic.loc[lic['LICENCE_TYPE_NAME'] == 'Spectrum']
specLic = lic.reset_index(drop=True, inplace=False)

# Create a series of Spectrum Licence IDs
specLicID = specLic['LICENCE_NO']

# Change column type from object to 64 bit integer
specLic = specLic.astype({'LICENCE_NO':'int64'})

# Filter out non-mid-band licences
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == 'AWL - FSS Only')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == 'AWL - Standard')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == '26 GHz Band')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == '20 GHz Defence Band')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == '30 GHz Defence Band')]
specLic = specLic.reset_index(drop=True, inplace=False)

# Save to file
specLic.to_csv('ds/spectrumLicences.csv', index=False)
specLic

,LICENCE_NO,CLIENT_NO,LICENCE_TYPE_NAME,LICENCE_CATEGORY_NAME
0,10143110,1136980.0,Spectrum,2 GHz Band
1,10143136,1103274.0,Spectrum,2 GHz Band
2,10143466,1104504.0,Spectrum,2 GHz Band
3,10143562,1103276.0,Spectrum,2 GHz Band
4,10231258,1103276.0,Spectrum,1800 MHz Band
...,...,...,...,...
61,9469870,1149289.0,Spectrum,2.5 GHz Band
62,9469871,1103275.0,Spectrum,2.5 GHz Band
63,9469878,1103275.0,Spectrum,2.5 GHz Band
64,9599659,1103275.0,Spectrum,2.3 GHz Band


## Identify Frequency Ranges and Locations:

In [7]:
freq = pd.read_csv('auth_spectrum_freq.csv')

# Remove irrelevant data
freq = freq.drop(columns=['AREA_CODE'])

# Convert Hz to MHz
#freq = freq.div(10000000, axis=['LW_FREQUENCY_START', 'LW_FREQUENCY_END', 'UP_FREQUENCY_START', 'UP_FREQUENCY_END'])
freq['LW_FREQUENCY_START'] = (freq['LW_FREQUENCY_START'] / 1000000).round(3)
freq['LW_FREQUENCY_END'] = (freq['LW_FREQUENCY_END'] / 1000000).round(3)
freq['UP_FREQUENCY_START'] = (freq['UP_FREQUENCY_START'] / 1000000).round(3)
freq['UP_FREQUENCY_END'] = (freq['UP_FREQUENCY_END'] / 1000000).round(3)

# Save to file
freq.to_csv('ds/frequencyList.csv', index='False')
freq

,LICENCE_NO,AREA_NAME,LW_FREQUENCY_START,LW_FREQUENCY_END,UP_FREQUENCY_START,UP_FREQUENCY_END
0,10006309,Regional South Australia,1710.0,1712.5,1805.0,1807.5
1,10143110,Perth,1960.0,1970.0,2150.0,2160.0
2,10143110,Sydney,1920.0,1935.0,2110.0,2125.0
3,10143110,Adelaide,1960.0,1970.0,2150.0,2160.0
4,10143110,Melbourne,1920.0,1935.0,2110.0,2125.0
...,...,...,...,...,...,...
1440,9599659,2.3 GHz Band,2302.0,2337.0,NaN,NaN
1441,9619844,Sydney,1740.0,1755.0,1835.0,1850.0
1442,9619844,Merge: Brisbane + Merge: Adelaide + Perth,1745.0,1755.0,1840.0,1850.0
1443,9619844,Melbourne,1740.0,1755.0,1835.0,1850.0


## Combine and Filter Datasets

In [10]:
# Isolate frequency and location data for spectrum licences
specFreq = freq[freq['LICENCE_NO'].isin(specLic['LICENCE_NO'])]
specFreq = specFreq.reset_index(drop=True, inplace=False)

# Add bandwidth values
specFreq['LW_BW'] = specFreq['LW_FREQUENCY_END'] - specFreq['LW_FREQUENCY_START']
specFreq['UP_BW'] = specFreq['UP_FREQUENCY_END'] - specFreq['UP_FREQUENCY_START']
specFreq['USAGE'] = specFreq['LW_BW'] + specFreq['UP_BW']

# Move LW_BW column
names = ['LICENCE_NO', 'AREA_NAME', 'LW_FREQUENCY_START', 'LW_FREQUENCY_END', 'LW_BW', 'UP_FREQUENCY_START', 'UP_FREQUENCY_END', 'UP_BW', 'USAGE']
specFreq = specFreq.reindex(columns=names)

# Add band categories
specFreq['BAND'] = (specFreq['LW_FREQUENCY_START'] / 1000).round(1)

# Save to file
specFreq.to_csv('ds/spectrumLicencesFreq.csv', index=False)
specFreq

,LICENCE_NO,AREA_NAME,LW_FREQUENCY_START,LW_FREQUENCY_END,LW_BW,UP_FREQUENCY_START,UP_FREQUENCY_END,UP_BW,USAGE,BAND
0,10143110,Perth,1960.0,1970.0,10.0,2150.0,2160.0,10.0,20.0,2.0
1,10143110,Sydney,1920.0,1935.0,15.0,2110.0,2125.0,15.0,30.0,1.9
2,10143110,Adelaide,1960.0,1970.0,10.0,2150.0,2160.0,10.0,20.0,2.0
3,10143110,Melbourne,1920.0,1935.0,15.0,2110.0,2125.0,15.0,30.0,1.9
4,10143110,Brisbane,1960.0,1970.0,10.0,2150.0,2160.0,10.0,20.0,2.0
...,...,...,...,...,...,...,...,...,...,...
294,9599659,2.3 GHz Band,2365.0,2400.0,35.0,NaN,NaN,NaN,NaN,2.4
295,9599659,2.3 GHz Band,2302.0,2337.0,35.0,NaN,NaN,NaN,NaN,2.3
296,9619844,Sydney,1740.0,1755.0,15.0,1835.0,1850.0,15.0,30.0,1.7
297,9619844,Merge: Brisbane + Merge: Adelaide + Perth,1745.0,1755.0,10.0,1840.0,1850.0,10.0,20.0,1.7


## Visualise dataset:

In [11]:
import plotly.express as px

fig = px.treemap(specFreq, path=[px.Constant('all'), 'AREA_NAME', 'BAND'], values='USAGE')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('treemap.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

